In [1]:
from InferenceNLP import *

2023-10-12 15:08:44.387709: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
inlp = InferenceNLP(
    sentclf_id = '/home/alpaco/csy/1005_sent_clf_with_clean',
    docclf_id = '/home/alpaco/csy/polyglot_for_document_clf_0926'
)

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at beomi/KoAlpaca-Polyglot-5.8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at beomi/KoAlpaca-Polyglot-5.8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
sample = '/home/alpaco/zenodo/handmadedata/call_2.wav'
result_sent, result_doc = inlp.run(sample)

In [10]:
len(result_sent)

25

In [13]:
from inference_AUDIO import *

# inference를 실행할 디바이스를 지정
device = "cuda:0" if torch.cuda.is_available() else 'cpu'

# 불러올 멀티모달 모델 클래스와 체크포인트를 지정
# final_model_class = MultiModal_mk1()
# final_checkpoint_path = ''
# mm_model = load_torch_model(final_model_class, final_checkpoint_path).to(device)
image_model_path = '/home/alpaco/mel_spec_approach/khrdatatest/mk1_khrdatatest_lr0001_3.pt'
audio_model_path = '/home/alpaco/KHR/MM_whole/checkpoints/new_my_audio_classifier_last_088_88.pt'
mm_model = MultiModal_mk1(image_model_path, audio_model_path).to(device)
mm_model.eval()
HUBERT = load_hubert()

epoch 3 val_loss 0.09617409097390767 val_accuracy 0.9742491525869091
epoch 88 val_loss tensor(0.0318, device='cuda:0') val_accuracy 0.9469742754289618
[facebook/hubert-base-ls960 loaded]
...............to cuda:0


In [15]:
# 4초마다 하나씩 inference
wav_path = '/home/alpaco/zenodo/handmadedata/call_2.wav'
wavs = preprocess_audio(wav_path)

predictions = []
for wav in wavs:
    mels, audio_value_sixths, audio_value_lasts = crop_to_data(wav, HUBERT)
    logit = mm_model(mels.unsqueeze(0).to(device), audio_value_lasts.to(device)).detach().cpu().numpy() # audio_value_sixths, # in shape of (1, 2)
    prediction = np.argmax(logit, axis=1)[0]
    predictions.append(prediction)

# 파이널모델 불러오기

In [1]:
from inference_AUDIO import *
image_model_path = '/home/alpaco/mel_spec_approach/khrdatatest/mk1_khrdatatest_lr0001_12.pt'
audio_model_path = '/home/alpaco/KHR/MM_whole/checkpoints/new_my_audio_classifier_last_088_88.pt'
# mm_model = MultiModal_mk1(image_model_path, audio_model_path).to(device)
final_model_class = MultiModal_mk1
final_checkpoint_path = '/home/alpaco/KHR/MM_whole/mm_checkpoints/mm_focal_4.pt'
model = load_torch_model(final_model_class, final_checkpoint_path, image_model_path=image_model_path, audio_model_path=audio_model_path)
model = model.to(device)

epoch 12 val_loss 0.05103678643046057 val_accuracy 0.9842867278030323
epoch 88 val_loss tensor(0.0318, device='cuda:0') val_accuracy 0.9469742754289618
epoch 4 val_loss tensor(0.0048, device='cuda:0') val_accuracy 0.9933520771474368


In [2]:
def load_hubert(hubert_name="facebook/hubert-base-ls960", device=device):
    hubert = HubertModel.from_pretrained("facebook/hubert-base-ls960")
    print(f'[{hubert_name} loaded]')
    hubert = hubert.to(device)
    print('...............to', device)
    return hubert

ubert_model = load_hubert()

[facebook/hubert-base-ls960 loaded]
...............to cuda:0


In [3]:
def audio_load_pad_mel_lbrs(wav, crop_from=None, target_sec=4, sample_rate=16000):
    # Calculate target length in samples
    if type(wav) == str:
        wav, _ = librosa.load(wav, sr = 16000)
    target_length_samples = int(target_sec * sample_rate)
    wav = np.array(wav.squeeze())
    # wav = wav[crop_from : crop_from+target_length_samples]
    current_length_samples = wav.shape[-1]
    padding_samples = target_length_samples - current_length_samples
    # Pad the waveform with zeros
    padding_waveform = np.zeros((padding_samples), dtype=np.int32)
    padded_waveform = np.concatenate((wav, padding_waveform))

    spect = librosa.feature.melspectrogram(y=padded_waveform, sr=16000, n_fft=2048, n_mels = 128, win_length=400, hop_length=512)
    data_spec = librosa.power_to_db(spect, ref=np.max)
    mel_spectrogram_np = np.array(data_spec, np.float32)
    mel_spectrogram = torch.Tensor(mel_spectrogram_np)

    return mel_spectrogram # (126, 128)의 텐서

def crop_to_data(audio_crop, hubert_model=ubert_model):
    assert audio_crop.size(1) == 64000, 'input must be A cropped audio in tensor with a length of 64000'

    mels = audio_load_pad_mel_lbrs(audio_crop, crop_from=0)

    audio_value = hubert_model(audio_crop.to(device), output_hidden_states=True)
    audio_value_sixths = audio_value.hidden_states[6].squeeze().mean(0) # .detach().cpu().numpy()
    audio_value_lasts = audio_value.last_hidden_state.squeeze().mean(0)

    return mels.unsqueeze(0), audio_value_sixths.unsqueeze(0), audio_value_lasts.unsqueeze(0)

In [4]:
wav = torch.tensor(np.random.normal(size=(1, 64000))).float()
mels, audio_value_sixths, audio_value_lasts = crop_to_data(wav, ubert_model)

In [5]:
mels.shape, audio_value_sixths.shape, audio_value_lasts.shape

(torch.Size([1, 128, 126]), torch.Size([1, 768]), torch.Size([1, 768]))

In [13]:
logit = model(mels.unsqueeze(0).to(device), audio_value_lasts.to(device)).detach().cpu().numpy()

In [6]:
with torch.no_grad():
    logit_nograd = model(mels.unsqueeze(0).to(device), audio_value_lasts.to(device)).detach().cpu().numpy()

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 64])

In [ ]:
# eval() 안 켜면 nograd 켜든 안켜든
# Expected more than 1 value per channel when training, got input size torch.Size([1, 64])
# 배치놂 때문에 하나씩은 넣을 수 없음.

In [14]:
# eval() 만 켰을 때
logit

array([[ 4.095861 , -5.3941493]], dtype=float32)

In [16]:
# eval() 을 켠 다음 nograd
logit_nogradandeval

array([[ 4.095861 , -5.3941493]], dtype=float32)